# Creating test cases
This notebook takes a folder of notebooks and turns them into a jsonl file in the format human_eval expects.

In [1]:
import json
import os
import warnings

In [2]:
source_notebook_directory = '../test_cases/' # must end with /
# Specify the filename to save the .jsonl file
target_jsonl_filename = '../data/human-eval-bia.jsonl'

In [3]:
list_of_cases = []


# List all files in the current directory
files = os.listdir(source_notebook_directory)

# Iterate through the files and print names ending with .ipynb
for file in files:
    if file.endswith('.ipynb'):
        notebook_filename = source_notebook_directory + file
        
        # Load and parse the notebook
        with open(notebook_filename, 'r') as file:
            notebook = json.load(file)
        
        task_id = notebook_filename
        prompt = None
        canonical_solution = None
        entry_point = None
        test = None
        
        # Iterate through the cells and print the source of code cells
        for cell in notebook['cells']:
            if cell['cell_type'] == 'code':
                # Joining the lines of code for better readability
                code = ''.join(cell['source'])
                # print('\n\nCODE\n\n',code)
        
                if code.startswith('check('):
                    entry_point = code.strip().replace("check(","").replace(")","").strip()
                elif '"""' in code:
                    temp = code.split('"""')
                    canonical_solution = temp[-1]
                    temp[-1] = ""
                    prompt = '"""'.join(temp)
                elif 'def check(' in code:
                    test = code 
                elif len(code.strip()) == 0:
                    pass
                else:
                    sample = code[:20]
                    warnings.warn(f"I had issues reading a cell in {task_id} starting with ")
                    
        if prompt is None:
            warnings.warn(f"Couldn't extract prompt from {task_id}.")
        elif canonical_solution is None:
            warnings.warn(f"Couldn't extract canonical_solution from {task_id}.")
        elif entry_point is None:
            warnings.warn(f"Couldn't extract entry_point from {task_id}.")
        
        test_case = {
            'task_id':task_id,
            'prompt':prompt,
            'canonical_solution':canonical_solution,
            'entry_point':entry_point,
            'test':test
        }

        print(test_case)
        list_of_cases.append(test_case)

{'task_id': '../test_cases/apply_otsu_threshold_and_count_postiive_pixels.ipynb', 'prompt': 'def apply_otsu_threshold_and_count_postiive_pixels(image):\n    """\n    Takes an image as numpy array, applies Otsu\'s threshold method to it to create a binary image and \n    counts the positive pixels and returns this number.\n    """', 'canonical_solution': '\n    import skimage\n    import numpy as np\n    binary_image = image > skimage.filters.threshold_otsu(image)\n\n    result = np.sum(binary_image)\n\n    return result', 'entry_point': 'apply_otsu_threshold_and_count_postiive_pixels', 'test': 'def check(candidate):\n    import numpy as np\n    \n    assert candidate(np.asarray([\n        [0,0,0,0,0],\n        [1,1,1,0,0],\n        [1,1,1,0,0],\n        [1,0,0,0,0],\n        [0,0,0,1,0],\n    ])) == 8\n\n    assert candidate(np.asarray([\n        [0,0,0,0,0],\n        [0,1,0,0,0],\n        [1,2,1,0,0],\n        [0,1,3,4,0],\n        [0,1,4,1,0],\n    ])) == 4\n\n    assert candidate(np

C:\Users\rober\AppData\Local\Temp\ipykernel_41440\2539596063.py:42: UserWarning: I had issues reading a cell in ../test_cases/workflow_watershed_segmentation_correction_measurement.ipynb starting with 
  warnings.warn(f"I had issues reading a cell in {task_id} starting with ")


In [4]:
# Open the file in write mode and save the dictionaries
with open(target_jsonl_filename, 'w') as file:
    for dictionary in list_of_cases:
        # Convert dictionary to a JSON formatted string and write it
        json_str = json.dumps(dictionary)
        file.write(json_str + '\n')

print(f'Data successfully saved to {target_jsonl_filename}.')

Data successfully saved to ../data/human-eval-bia.jsonl.


In [5]:
filename = source_notebook_directory + "readme.md"
n = len(list_of_cases)
header = f"""
# List of use-cases
This list of {n} use-cases is auto-generated. Do not modify this file.

"""

with open(filename, 'w') as file:
    file.write(header)
    for test_case in list_of_cases:

        name = test_case["entry_point"]
        task = test_case["task_id"].replace("../test_cases/", "")

        description = test_case["prompt"].split('"""')[-2]
        
        file.write(f"""
* [{name}]({task}): {description}
        """)